In [59]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import datetime, time

today = datetime.datetime.today()

In [8]:
urlpage =  'https://www.myfico.com/loan-center/home-mortgage-rate-comparison/default.aspx'

# query the website and return the html to the variable 'page'
page = requests.get(urlpage)

# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page.content, 'html.parser')


In [10]:
print(soup)

<!DOCTYPE doctype html>

<html lang="en-US">
<head id="ctl00_ctl00_Head1"><title>
	myFICO Loan Center: Free Info on Loans &amp; Interest Rates
</title><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="myFICO Loan Center provides free information on home loans, refinance, home equity and interest rates. Get offers from trusted lenders for your situation." name="Description"/>
<meta content="mortgage rates, mortgage payment, compare mortgage rates" name="Keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="en-us" http-equiv="Content-Language"/>
<meta content="false" http-equiv="imagetoolbar"/>
<meta content="app-id=com.myfico" name="google-play-app"/>
<link href="https://www.myfico.com/static/images/icons/com.myfico.png" rel="apple-touch-icon"/>
<meta content="Copyright (c) 2001-2020 Fair Isaac Corporation. All rights reserved." name="copyri

In [13]:
# div[@id="ratewidget"] / ul[@id="loanselect"] = one li element for each tab
# Results are in the div "ratewidget"
divMain = soup.find('div', attrs={'id': 'ratewidget'})
print(divMain)

<div id="ratewidget">
<h3 id="ctl00_ctl00_cphMainContent_cphMainContent_informaRateCon_informaRateConJS_hAsOfLine"><span id="ctl00_ctl00_cphMainContent_cphMainContent_informaRateCon_informaRateConJS_lblAsOfText">Mortgage rates as of </span>
<span autoupdateaftercallback="true" id="ctl00_ctl00_cphMainContent_cphMainContent_informaRateCon_informaRateConJS_lblAsOf">May 1, 2020</span></h3>
<ul id="loanselect">
<li class="current" id="Y30FConfCombMtg_LI">
<a href="#" id="Y30FConfCombMtg" onclick="loanTypeClick(this.id);return false;">30-year fixed</a>
</li>
<li class="" id="Y15FConfCombMtg_LI">
<a href="#" id="Y15FConfCombMtg" onclick="loanTypeClick(this.id);return false;">15-year fixed</a>
</li>
<li class="" id="Y7ARMMtg_LI">
<a href="#" id="Y7ARMMtg" onclick="loanTypeClick(this.id);return false;">7/1 ARM</a>
</li>
<li class="" id="Y3ARMConfCombMtg_LI">
<a href="#" id="Y3ARMConfCombMtg" onclick="loanTypeClick(this.id);return false;">3/1 ARM</a>
</li>
<li class="" id="Y1ARMMtg_LI">
<a href=

In [14]:
tabs = divMain.find_all('li')
print(f'Tabs: {len(tabs)}')

Tabs: 5


In [16]:
tab1 = divMain.find('li') 
print(tab1)

<li class="current" id="Y30FConfCombMtg_LI">
<a href="#" id="Y30FConfCombMtg" onclick="loanTypeClick(this.id);return false;">30-year fixed</a>
</li>


In [18]:
tab1_data = soup.find('table', attrs={'id': 'tblSelectedRateData'})
print(tab1_data)

<table id="tblSelectedRateData" summary="Current interest rates for different FICO® score ranges">
<thead>
<tr>
<th scope="col">FICO® score</th>
<th scope="col">APR <a href="#" id="APRNote" title="More about these rates">[?]</a></th>
<th scope="col">Monthly payment<span id="ctl00_ctl00_cphMainContent_cphMainContent_informaRateCon_informaRateConJS_lblARMPointer" style="display:none;"> * </span></th>
</tr>
</thead>
<tbody>
<tr>
<th id="range0" scope="row">760-850</th>
<td id="rate0">3.009%</td>
<td id="payment0">$1,266</td>
</tr>
<tr>
<th id="range1" scope="row">700-759</th>
<td id="rate1">3.231%</td>
<td id="payment1">$1,302</td>
</tr>
<tr>
<th id="range2" scope="row">680-699</th>
<td id="rate2">3.408%</td>
<td id="payment2">$1,332</td>
</tr>
<tr>
<th id="range3" scope="row">660-679</th>
<td id="rate3">3.622%</td>
<td id="payment3">$1,368</td>
</tr>
<tr>
<th id="range4" scope="row">640-659</th>
<td id="rate4">4.052%</td>
<td id="payment4">$1,441</td>
</tr>
<tr>
<th id="range5" scope="ro

In [35]:
table = soup.find('table', attrs={'id': 'tblSelectedRateData'})
table_rows = table.find_all('tr')

# tbody: th has the credit score and td have the APR and payment

res = []
for tr in table_rows:
    row = []
    
    # Get the FICO score first
    fico_score = tr.find('th').text.strip()
    row.append(fico_score)
    
    # Then get the two row values
    td = tr.find_all('td')
    # Don't use a list compression as it returns another array
    # temprow = [tr.text.strip() for tr in td if tr.text.strip()]
    
    for x in td:
        row.append(x.text.strip())
    
    res.append(row)

df = pd.DataFrame(res)#, columns=["fico_score_range", "apr", "monthly_payment"])
print(df)

             0       1       2
0  FICO® score    None    None
1      760-850  3.009%  $1,266
2      700-759  3.231%  $1,302
3      680-699  3.408%  $1,332
4      660-679  3.622%  $1,368
5      640-659  4.052%  $1,441
6      620-639  4.598%  $1,538


In [40]:
table = soup.find('table', attrs={'id': 'tblSelectedRateData'})
table_rows = table.find_all('tr')

# tbody: th has the credit score and td have the APR and payment

res = []
for tr in table_rows:
    row = []
    
    # Get the FICO score first
    fico_score = tr.find('th').text.strip()
    row.append(fico_score)
    
    # Then get the two row values
    td = tr.find_all('td')
    # Don't use a list compression as it returns another array
    # temprow = [tr.text.strip() for tr in td if tr.text.strip()]
    
    for x in td:
        row.append(x.text.strip().replace('$', '').replace(',', '').replace('%', ''))    
    res.append(row)

df = pd.DataFrame(res, columns=["fico_score_range", "apr", "monthly_payment"])

# drop the first row
df = df.iloc[1:]

print(df)

  fico_score_range    apr monthly_payment
1          760-850  3.009            1266
2          700-759  3.231            1302
3          680-699  3.408            1332
4          660-679  3.622            1368
5          640-659  4.052            1441
6          620-639  4.598            1538


In [61]:
table = soup.find('table', attrs={'id': 'tblSelectedRateData'})
table_rows = table.find_all('tr')

# tbody: th has the credit score and td have the APR and payment

res = []
for tr in table_rows:
    row = []
    
    # Get the FICO score first
    fico_score = tr.find('th').text.strip()
    row.append(fico_score)
    
    # Then get the two row values
    td = tr.find_all('td')
    # Don't use a list compression as it returns another array
    # temprow = [tr.text.strip() for tr in td if tr.text.strip()]
    
    for x in td:
        row.append(x.text.strip().replace('$', '').replace(',', '').replace('%', ''))    
    res.append(row)

df = pd.DataFrame(res, columns=["fico_score_range", "apr", "monthly_payment"])

# drop the first row
df = df.iloc[1:]

# pandas treats regex a bit weird - you have to add the '()' which says "For each capture group"
df['start_range'] = df['fico_score_range'].str.extract(pat='(^\d{3})', expand=False)
df['end_range'] = df['fico_score_range'].str.extract('(\d{3}$)', expand=True)
df['SnapshotDate']= pd.to_datetime(today) 
df.head()

,fico_score_range,apr,monthly_payment,start_range,end_range,SnapshotDate
1,760-850,3.009,1266,760,850,2020-05-04 07:37:48.359502
2,700-759,3.231,1302,700,759,2020-05-04 07:37:48.359502
3,680-699,3.408,1332,680,699,2020-05-04 07:37:48.359502
4,660-679,3.622,1368,660,679,2020-05-04 07:37:48.359502
5,640-659,4.052,1441,640,659,2020-05-04 07:37:48.359502
